# OpenAI API + [Weights & Biases](https://wandb.ai/site) Integration

Learn more about LLMOps at https://wandb.me/prompts

> 💡 **Beta Integration**: This is a new feature, and we're actively working on making this better. Please reach out if you have any feedback — contact@wandb.com

OpenAI’s API gives practitioners access to GPT-4, an incredibly powerful natural language model that can be applied to virtually any task that involves understanding or generating natural language.

## Log OpenAI API calls in 1 line of code
With just 1 line of code you can now automatically log inputs and outputs from the OpenAI Python SDK to Weights & Biases! 

![](./open_ai_autolog.png)

To get started, pip install the `wandb` library, then follow the steps below:

### 1. Import autolog and initialise it
First, import `autolog` from `wandb.integration.openai` and initialise it.  

In [ ]:
import os
import openai
from wandb.integration.openai import autolog

autolog({"project":"gpt5"})

You can optionally pass a dictionary with argument that `wandb.init()` accepts to `autolog`. This includes a project name, team name, entity, and more. For more information about [`wandb.init`](https://docs.wandb.ai/ref/python/init), see the API Reference Guide.

### 2. Call the OpenAI API
Each call you make to the OpenAI API will now be logged to Weights & Biases automatically.

In [ ]:
os.environ["OPENAI_API_KEY"] = "XXX"

chat_request_kwargs = dict(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers"},
        {"role": "user", "content": "Where was it played?"},
    ],
)
response = openai.ChatCompletion.create(**chat_request_kwargs)

### 3. View your OpenAI API inputs and responses

Click on the Weights & Biases [run](https://docs.wandb.ai/guides/runs) link generated by `autolog` in **step 1**. This will redirect you to your project workspace in the W&B App.

Select a run you created to view the trace table, trace timeline and the model architecture of the OpenAI LLM used.

### 4. Disable autolog
We recommend that you call `disable()` to close all W&B processes when you are finished using the OpenAI API.

In [ ]:
autolog.disable()

Now your inputs and completions will be logged to Weights & Biases, ready for analysis or to be shared with colleagues.

## Log OpenAI fine-tunes to W&B

If you use OpenAI's API to [fine-tune GPT-3](https://beta.openai.com/docs/guides/fine-tuning), you can now use the W&B integration to track experiments, models, and datasets in your central dashboard.

![](./open_ai_api.png)

All it takes is one line: `openai wandb sync`

## ✨ Check out interactive examples

* [Demo Colab](http://wandb.me/openai-colab)
* [Report - GPT-3 Exploration and Fine-Tuning Tips](http://wandb.me/openai-report)

## 🎉 Sync your fine-tunes with one line!

Make sure you are using latest version of openai and wandb.

In [ ]:
!pip install --upgrade openai wandb

Then sync your results from the command line or from your script.

### Command Line

In [ ]:
# one line command
!openai wandb sync

# passing optional parameters
!openai wandb sync --help

### Python

In [ ]:
from openai.wandb_logger import WandbLogger

# one line command
WandbLogger.sync()

# passing optional parameters
WandbLogger.sync(
    id=None,
    n_fine_tunes=None,
    project="GPT-3",
    entity=None,
    force=False,
    **kwargs_wandb_init
)

We scan for new completed fine-tunes and automatically add them to your dashboard.

![](./open_ai_auto_scan.png)

In addition your training and validation files are logged and versioned, as well as details of your fine-tune results. This let you interactively explore your training and validation data.

![](./open_ai_validation_files.png)

## ⚙️ Optional arguments

| Argument                 | Description                                                                                                               |
| ------------------------ | ------------------------------------------------------------------------------------------------------------------------- |
| -i ID, --id ID           | The id of the fine-tune (optional)                                                                                        |
| -n N, --n\_fine\_tunes N | Number of most recent fine-tunes to log when an id is not provided. By default, every fine-tune is synced.                |
| --project PROJECT        | Name of the project where you're sending runs. By default, it is "GPT-3".                                                 |
| --entity ENTITY          | Username or team name where you're sending runs. By default, your default entity is used, which is usually your username. |
| --force                  | Forces logging and overwrite existing wandb run of the same fine-tune.                                                    |
| \*\*kwargs\_wandb\_init  | In python, any additional argument is directly passed to [`wandb.init()`](https://docs.wandb.ai/ref/python/init)                    |

## 🔍 Inspect sample predictions

Use [Tables](https://docs.wandb.ai/guides/data-vis) to better visualize sample predictions and compare models.

![](./open_ai_inspect_sample.png)

Create a new run:

In [ ]:
run = wandb.init(project="GPT-3", job_type="eval")

Retrieve a model id for inference.

You can use automatically logged artifacts to retrieve your latest model:

In [ ]:
artifact_job = run.use_artifact("ENTITY/PROJECT/fine_tune_details:latest")
fine_tuned_model = artifact_job.metadata["fine_tuned_model"]

You can also retrieve your validation file:

In [ ]:
artifact_valid = run.use_artifact("ENTITY/PROJECT/FILENAME:latest")
valid_file = artifact_valid.get_path("FILENAME").download()

Perform some inferences using OpenAI API:

In [ ]:
# perform inference and record results
my_prompts = ["PROMPT_1", "PROMPT_2"]
results = []
for prompt in my_prompts:
    res = openai.Completion.create(model=fine_tuned_model,
                                   prompt=prompt,
                                   ...)
    results.append(res["choices"][0]["text"])

Log your results with a Table:

In [ ]:
table = wandb.Table(columns=['prompt', 'completion'],
                    data=list(zip(my_prompts, results)))

## 📕 Resources

* [OpenAI Fine-tuning Documentation](https://beta.openai.com/docs/guides/fine-tuning) is very thorough and contains many useful tips
* [Demo Colab](http://wandb.me/openai-colab)
* [Report - GPT-3 Exploration & Fine-tuning Tips](http://wandb.me/openai-report)